### Select Twogram Threegram Fourgram Fivegram Sixgram According To Word Usage Limit

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from nltk import ngrams
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Intersect"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0 native word start index
word_end = 10000  # 1000 native word end index

# word usage
word_usage_limit = False
min_word_use_num = 1
max_word_use_num = 3

# sentence check
twogram_sentence_check = False  # True, False
threegram_sentence_check = False
fourgram_sentence_check = False
fivegram_sentence_check = False
sixgram_sentence_check = False

In [4]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by="frequency", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Talk Time/Talk Time 4/Result/1-Select Twogram Threegram According To Word Usage Limit"

Path(path).mkdir(parents=True, exist_ok=True)

In [7]:
#disable_video_id_list = ["H6E6N70jYqI","BoaYsdPtJYA","HUZINsU40Fk","Et3diPcEmfY"]
disable_video_id_list = []

In [8]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [9]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
4995,istemeyiz,12450
4996,iniş,12450
4997,bakarız,12447
4998,say,12445


In [10]:
word_list = df_word_select["word"].to_list()
len(word_list)

5000

In [11]:
if twogram_sentence_check:
    df_twogram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Two_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_twogram_sent.rename(columns={"two_gram":"twogram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_twogram = df_twogram_sent.loc[:,["twogram","frequency"]]
else:
    df_twogram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")  
    df_twogram = df_twogram.loc[:,["twogram","frequency"]]

df_twogram

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [12]:
df_twogram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Twogram_Used.xlsx")
#df_twogram_used

In [13]:
twogram_used_list = list(set(df_twogram_used["twogram"]))

In [14]:
# Twogram Func
d_list2  = df_twogram.iloc[:,0].values.tolist()

resultlist2 = []
manager = multiprocessing.Manager()
resultlist2 = manager.list()

def word_in_wordgroup(d_list2):
    mergelist2 = []
    try:
        word = d_list2.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist2.append(word[j])
            if len(mergelist2) == len(word):
                    resultlist2.append(d_list2)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list2) # string_word liste

result_list2 = list(resultlist2)
df_result2 = pd.DataFrame(result_list2, columns=[0])  # add columns parameter for empty result
df_result2 = df_result2.rename(columns = {0: "twogram"})
df_merge2 = pd.merge(df_result2, df_twogram, how="left", on="twogram")
df_merge_result2 = df_merge2.sort_values(by="frequency", ascending=False)
df_merge_result2.drop_duplicates(inplace=True)
df_merge_result2.reset_index(drop=True, inplace=True)
df_twogram_select = df_merge_result2
df_twogram_select

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
884788,rahat vermiyorsun,6
884789,yasa bizim,6
884790,kadını mutlaka,6
884791,sence aynısını,6


In [15]:
df_twogram_select = df_twogram_select[~df_twogram_select["twogram"].isin(twogram_used_list)]
df_twogram_select

,twogram,frequency
3,teşekkür ederim,370619
6,hayır hayır,266457
8,öyle mi,263184
11,ne yapıyorsun,236176
12,bu kadar,215204
...,...,...
884788,rahat vermiyorsun,6
884789,yasa bizim,6
884790,kadını mutlaka,6
884791,sence aynısını,6


In [16]:
if word_usage_limit:
    df_twogram_word_usage = word_usage_result(word_list, df_twogram_select, "twogram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_twogram_word_usage = df_twogram_select.head(200)

df_twogram_word_usage

,twogram,frequency
3,teşekkür ederim,370619
6,hayır hayır,266457
8,öyle mi,263184
11,ne yapıyorsun,236176
12,bu kadar,215204
...,...,...
256,bu işi,31396
257,hatırlıyor musun,31310
258,orospu çocuğu,31262
259,çok teşekkürler,31230


In [17]:
word_count_result(df_twogram_word_usage, ["twogram"], set_condition=False)

,word,word_count
0,bir,37
1,ne,28
2,bu,19
3,çok,8
4,ben,7
...,...,...
189,seviyorum,1
190,yapacağız,1
191,yapacaksın,1
192,yani,1


In [18]:
df_twogram_word_usage.to_excel(f"{lang_folder.capitalize()}_Twogram_Selected_With_{word_end}_Word.xlsx", index=False)

In [19]:
if threegram_sentence_check:
    df_threegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Three_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_threegram_sent.rename(columns={"three_gram":"threegram"}, inplace=True)  # ext. Not: Two_Gram_And_Sentence.csv convert to Two_Gram_And_Sentence_All.csv
    df_threegram = df_threegram_sent.loc[:,["threegram","frequency"]]
else:
    df_threegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")  
    df_threegram = df_threegram.loc[:,["threegram","frequency"]]

df_threegram

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [20]:
df_threegram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Threegram_Used.xlsx")
#df_threegram_used

In [21]:
threegram_used_list = list(set(df_threegram_used["threegram"]))

In [22]:
# Threegram Func
d_list3  = df_threegram.iloc[:,0].values.tolist()

resultlist3 = []
manager = multiprocessing.Manager()
resultlist3 = manager.list()

def word_in_wordgroup(d_list3):
    mergelist3 = []
    try:
        word = d_list3.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist3.append(word[j])
            if len(mergelist3) == len(word):
                    resultlist3.append(d_list3)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list3) # string_word liste

result_list3 = list(resultlist3)
df_result3 = pd.DataFrame(result_list3, columns=[0])  # add columns parameter for empty result
df_result3 = df_result3.rename(columns = {0: "threegram"})
df_merge3 = pd.merge(df_result3, df_threegram, how="left", on="threegram")
df_merge_result3 = df_merge3.sort_values(by="frequency", ascending=False)
df_merge_result3.drop_duplicates(inplace=True)
df_merge_result3.reset_index(drop=True, inplace=True)
df_threegram_select = df_merge_result3
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
1217933,kolay kontrol altında,10
1217934,fakat katil o,10
1217935,fakat katılmak isterseniz,10
1217936,kolay hale getireyim,10


In [23]:
df_threegram_select = df_threegram_select[~df_threegram_select["threegram"].isin(threegram_used_list)]
df_threegram_select

,threegram,frequency
0,bir şey yok,113165
3,her şey yolunda,75968
4,başka bir şey,75193
5,hayır hayır hayır,69443
6,bir şey değil,68498
...,...,...
1217933,kolay kontrol altında,10
1217934,fakat katil o,10
1217935,fakat katılmak isterseniz,10
1217936,kolay hale getireyim,10


In [24]:
if word_usage_limit:
    df_threegram_word_usage = word_usage_result(word_list, df_threegram_select, "threegram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_threegram_word_usage = df_threegram_select.head(200)

df_threegram_word_usage

,threegram,frequency
0,bir şey yok,113165
3,her şey yolunda,75968
4,başka bir şey,75193
5,hayır hayır hayır,69443
6,bir şey değil,68498
...,...,...
216,ne var ne,10026
217,ben de bunu,10004
218,seni görmek güzel,9941
219,tabii ki hayır,9936


In [25]:
word_count_result(df_threegram_word_usage, ["threegram"], set_condition=False)

,word,word_count
0,bir,66
1,ne,42
2,şey,41
3,bu,21
4,var,18
...,...,...
177,yardım,1
178,tane,1
179,tanıştığımıza,1
180,teşekkür,1


In [26]:
df_threegram_word_usage.to_excel(f"{lang_folder.capitalize()}_Threegram_Selected_With_{word_end}_Word.xlsx", index=False)

In [27]:
if fourgram_sentence_check:
    df_fourgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Four_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fourgram_sent.rename(columns={"four_gram":"fourgram"}, inplace=True)  # ext. Not: Four_Gram_And_Sentence.csv convert to Four_Gram_And_Sentence_All.csv
    df_fourgram = df_fourgram_sent.loc[:,["fourgram","frequency"]]
else:
    df_fourgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")  
    df_fourgram = df_fourgram.loc[:,["fourgram","frequency"]]

df_fourgram

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [28]:
df_fourgram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Fourgram_Used.xlsx")
#df_fourgram_used

In [29]:
fourgram_used_list = list(set(df_fourgram_used["fourgram"]))

In [30]:
# Fourgram Func
d_list4  = df_fourgram.iloc[:,0].values.tolist()

resultlist4 = []
manager = multiprocessing.Manager()
resultlist4 = manager.list()

def word_in_wordgroup(d_list4):
    mergelist4 = []
    try:
        word = d_list4.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist4.append(word[j])
            if len(mergelist4) == len(word):
                    resultlist4.append(d_list4)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list4) # string_word liste

result_list4 = list(resultlist4)
df_result4 = pd.DataFrame(result_list4, columns=[0])  # add columns parameter for empty result
df_result4 = df_result4.rename(columns = {0: "fourgram"})
df_merge4 = pd.merge(df_result4, df_fourgram, how="left", on="fourgram")
df_merge_result4 = df_merge4.sort_values(by="frequency", ascending=False)
df_merge_result4.drop_duplicates(inplace=True)
df_merge_result4.reset_index(drop=True, inplace=True)
df_fourgram_select = df_merge_result4
df_fourgram_select

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
939127,kız arkadaş ve harika,7
939128,sana ilgi çekici gelmiyor,7
939129,benim evim soğuk mu,7
939130,onunla ne yapacağımı biliyor,7


In [31]:
df_fourgram_select = df_fourgram_select[~df_fourgram_select["fourgram"].isin(fourgram_used_list)]
df_fourgram_select

,fourgram,frequency
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
5,her şey yoluna girecek,17016
7,gereken bir şey var,15683
...,...,...
939127,kız arkadaş ve harika,7
939128,sana ilgi çekici gelmiyor,7
939129,benim evim soğuk mu,7
939130,onunla ne yapacağımı biliyor,7


In [32]:
if word_usage_limit:
    df_fourgram_word_usage = word_usage_result(word_list, df_fourgram_select, "fourgram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_fourgram_word_usage = df_fourgram_select.head(200)
    
df_fourgram_word_usage

,fourgram,frequency
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
5,her şey yoluna girecek,17016
7,gereken bir şey var,15683
...,...,...
208,sana bir şey söyleyeceğim,2278
209,önemli olan da bu,2278
210,o kadar kötü değil,2270
211,bu benim için çok,2256


In [33]:
word_count_result(df_fourgram_word_usage, ["fourgram"], set_condition=False)

,word,word_count
0,bir,97
1,şey,68
2,ne,34
3,var,26
4,bu,20
...,...,...
211,tane,1
212,yoluna,1
213,yoktur,1
214,tek,1


In [34]:
df_fourgram_word_usage.to_excel(f"{lang_folder.capitalize()}_Fourgram_Selected_With_{word_end}_Word.xlsx", index=False)

In [35]:
if fivegram_sentence_check:
    df_fivegram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Five_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_fivegram_sent.rename(columns={"five_gram":"fivegram"}, inplace=True)  # ext. Not: Five_Gram_And_Sentence.csv convert to Five_Gram_And_Sentence_All.csv
    df_fivegram = df_fivegram_sent.loc[:,["fivegram","frequency"]]
else:
    df_fivegram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")  
    df_fivegram = df_fivegram.loc[:,["fivegram","frequency"]]

df_fivegram

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [ ]:
df_fivegram_used = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Talk Time/Data/N Gram Used/Fivegram_Used.xlsx")
#df_fivegram_used

In [ ]:
fivegram_used_list = list(set(df_fivegram_used["fivegram"]))

In [36]:
# Fivegram Func
d_list5  = df_fivegram.iloc[:,0].values.tolist()

resultlist5 = []
manager = multiprocessing.Manager()
resultlist5 = manager.list()

def word_in_wordgroup(d_list5):
    mergelist5 = []
    try:
        word = d_list5.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist5.append(word[j])
            if len(mergelist5) == len(word):
                    resultlist5.append(d_list5)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list5) # string_word liste

result_list5 = list(resultlist5)
df_result5 = pd.DataFrame(result_list5, columns=[0])  # add columns parameter for empty result
df_result5 = df_result5.rename(columns = {0: "fivegram"})
df_merge5 = pd.merge(df_result5, df_fivegram, how="left", on="fivegram")
df_merge_result5 = df_merge5.sort_values(by="frequency", ascending=False)
df_merge_result5.drop_duplicates(inplace=True)
df_merge_result5.reset_index(drop=True, inplace=True)
df_fivegram_select = df_merge_result5
df_fivegram_select

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
251299,arka tarafında kendinize ait bir,10
251300,arka tarafında bir adam var,10
251301,boşa gitmesini istemeyiz değil mi,10
251302,kadar zaman içeride ne halt,10


In [ ]:
df_fivegram_select = df_fivegram_select[~df_fivegram_select["fivegram"].isin(fivegram_used_list)]
df_fivegram_select

In [37]:
if word_usage_limit:
    df_fivegram_word_usage = word_usage_result(word_list, df_fivegram_select, "fivegram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_fivegram_word_usage = df_fivegram_select.head(200)
    
df_fivegram_word_usage

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
195,bu konu hakkında konuşmak istemiyorum,816
196,altı beş dört üç iki,814
197,hiç böyle bir şey görmedim,813
198,istediğin başka bir şey var,812


In [38]:
word_count_result(df_fivegram_word_usage, ["fivegram"], set_condition=False)

,word,word_count
0,bir,116
1,şey,79
2,var,42
3,ne,32
4,bu,22
...,...,...
236,yapabileceğimiz,1
237,ters,1
238,tamam,1
239,söyleyeyim,1


In [39]:
df_fivegram_word_usage.to_excel(f"{lang_folder.capitalize()}_Fivegram_Selected_With_{word_end}_Word.xlsx", index=False)

In [ ]:
if sixgram_sentence_check:
    df_sixgram_sent = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/N Gram And Sentence/Six_Gram_And_Sentence.csv")  # ext. sentence and ngram
    df_sixgram_sent.rename(columns={"six_gram":"sixgram"}, inplace=True)  # ext. Not: Six_Gram_And_Sentence.csv convert to Six_Gram_And_Sentence_All.csv
    df_sixgram = df_sixgram_sent.loc[:,["sixgram","frequency"]]
else:
    df_sixgram = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Sixgram_Merge.csv")  
    df_sixgram = df_sixgram.loc[:,["sixgram","frequency"]]

df_sixgram

In [ ]:
# Sixgram Func
d_list6  = df_sixgram.iloc[:,0].values.tolist()

resultlist6 = []
manager = multiprocessing.Manager()
resultlist6 = manager.list()

def word_in_wordgroup(d_list6):
    mergelist6 = []
    try:
        word = d_list6.split()
    except:
        word = []
        #pass  disabled for non split value
    var1 = range(len(word))
    for j in var1:
        if word[j] in word_list:
            mergelist6.append(word[j])
            if len(mergelist6) == len(word):
                    resultlist6.append(d_list6)
                        
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(word_in_wordgroup, d_list6) # string_word liste

result_list6 = list(resultlist6)
df_result6 = pd.DataFrame(result_list6, columns=[0])  # add columns parameter for empty result
df_result6 = df_result6.rename(columns = {0: "sixgram"})
df_merge6 = pd.merge(df_result6, df_sixgram, how="left", on="sixgram")
df_merge_result6 = df_merge6.sort_values(by="frequency", ascending=False)
df_merge_result6.drop_duplicates(inplace=True)
df_merge_result6.reset_index(drop=True, inplace=True)
df_sixgram_select = df_merge_result6
df_sixgram_select

In [ ]:
if word_usage_limit:
    df_sixgram_word_usage = word_usage_result(word_list, df_sixgram_select, "sixgram", "frequency", min_word_use_num, max_word_use_num)
else:
    df_sixgram_word_usage = df_sixgram_select.head(200)
    
df_sixgram_word_usage

In [ ]:
word_count_result(df_sixgram_word_usage, ["sixgram"], set_condition=False)

In [ ]:
df_sixgram_word_usage.to_excel(f"{lang_folder.capitalize()}_Sixgram_Selected_With_{word_end}_Word.xlsx", index=False)

#### Copy Move And Delete

In [40]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_Selected_With_{word_end}_Word.xlsx")
output_file

['Turkish_Twogram_Selected_With_5000_Word.xlsx',
 'Turkish_Threegram_Selected_With_5000_Word.xlsx',
 'Turkish_Fourgram_Selected_With_5000_Word.xlsx',
 'Turkish_Fivegram_Selected_With_5000_Word.xlsx']

In [41]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [42]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass